### Exercício 1

Considere o exemplo de utilização do SCIP para encontrar $x$ e $y$ que minimizem $3x+4y$ satisfazendo as seguintes restrições:

$$
\left\{
\begin{array}{l}
5x + 6y \leq 200\\
2x + 80y \ge 150\\
0 \leq x \leq 100\\
0 \leq y \leq 100
\end{array}
\right.
$$

# Sobre linearidade

Os problemas que temos visto de Programação Inteira tem por objectivo calcular uma solução (ou a melhor solução, no caso de ser um problema de optimização) para um problema modelado como um conjunto de relações *lineares*, i.e., onde só são permitidas multiplicações em que um dos factores é constante.
O relaxamento desta condição torna o problema muito mais difícil, e muitos solvers não lhe conseguem dar resposta, ou apenas conseguem responder em situações muito limitadas.

In [1]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

x = solver.IntVar(0.0,100,"x")
y = solver.IntVar(0.0,100,"y")

solver.Add(5*x + 6*y <= 200)
solver.Add(2*x + 80*y >= 150)
solver.Add(x*y >=1) # dá erro de tipo e era o esperado !!
solver.Minimize(3*x + 4*y)

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print ("x = ",round(x.solution_value()))
    print ("y = ",round(y.solution_value()))
    print ("objectivo = ",round(solver.Objective().Value()))
else:
    print('No solution found.')

TypeError: 

Altere o programa de forma a acrescentar a restrição (não linear)  $ x y \ge 1$ e veja como ele se comporta.

Como se deve ter apercebido, o *wrapper* do SCIP assinala logo um erro de tipo, não perminido assim o produto de duas variáveis. 
O OR-Tools disponibiliza outras ferramentas de programação inteira que permitem este tipo de restrições, como é o caso do [CP-SAT solver](https://developers.google.com/optimization/cp/cp_solver).

## CP-SAT solver

O CP-SAT solver é uma ferramenta de programação inteira do OR-Tools que consegue lidar com a multiplicação de variáveis, embora de uma forma especial. 

A forma de interagir com o CP-SAT difere em alguns detalhes da do SCIP. Para ilustrar isso, apresentamos a seguir  a resolução comentada do exercício 1 em CP-SAT.

In [2]:
# Importar biblioteca
from ortools.sat.python import cp_model

# Cria o modelo CP-SAT
model = cp_model.CpModel()

# Cria as variáveis
x = model.NewIntVar(0, 100, 'x')
y = model.NewIntVar(0, 100, 'y')

# Adiciona as restrições ao modelo
model.Add(5*x + 6*y <= 200)
model.Add(2*x + 80*y >= 150)
model.Add(x*y >= 1)

# Define o objectivo
model.Minimize(3*x + 4*y)

# Cria um solver CP-SAT a solver and solves the model.
solver = cp_model.CpSolver()

# Invoca o solver com o modelo criado
status = solver.Solve(model)

# Interpreta os resultados
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))
    print ("objectivo = ", int(solver.ObjectiveValue()))
else:
    print('No solution found.')

TypeError: Not an number: y

Se tentar acrescentar a restrição $x y\ge 1$ através de `model.Add(x*y <= 1)` verá que obtém também um erro de tipo.
No CP-SAT a multiplicação de variáveis tem um tratamento especial.

A multiplicação de variáveis no CP-SAT não é uma operação como a multiplicação de um escalar por uma variável. Para cada multiplicação de variáveis é necessário criar uma variável adicional que representa essa multiplicação. É depois esta nova variável que entra nas restrições que são acrescentadas.

In [3]:
# Importar biblioteca
from ortools.sat.python import cp_model

# Cria o modelo CP-SAT
model = cp_model.CpModel()

# Cria as variáveis
x = model.NewIntVar(0, 100, 'x')
y = model.NewIntVar(0, 100, 'y')

# Adiciona as restrições ao modelo
model.Add(5*x + 6*y <= 200)
model.Add(2*x + 80*y >= 150)

# Cria uma variável adicional que representa a multiplicação das variáveis
xy = model.NewIntVar(0, 100*100, "xy")
model.AddMultiplicationEquality(xy, [x,y])  # xy = x*y
# Acescenta a restrição x*y>=1
model.Add(xy >= 1)

# Define o objectivo
model.Minimize(3*x + 4*y)

# Cria um solver CP-SAT a solver and solves the model.
solver = cp_model.CpSolver()

# Invoca o solver com o modelo criado
status = solver.Solve(model)

# Interpreta os resultados
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))
    print ("objectivo = ", int(solver.ObjectiveValue()))
else:
    print('No solution found.')

x = 1
y = 2
objectivo =  11


Tem [aqui](https://github.com/d-krupke/cpsat-primer) um breve tutorial do CP-SAT.

# Mais um problema de optimização linear

Um hospital trabalha todos os dias da semana e tem que ter um número mínimo de enfermeiros para assegurar o seu bom funcionamento. Os enfermeiros trabalham em turnos, e há 3 turnos: manhã, tarde ou noite.
As regras de funcionamento do hospital, quanto ao corpo de enfermagem, são as seguintes:

- Todos os turnos têm pelo menos N enfermeiros.
- Os turnos da manhã, por serem mais movimentados, têm no mínimo, mais 30% de enfermeiros do que é exigido nos restantes turnos.
- Um enfermeiro não pode trabalhar dois turnos seguidos.
- Todos os enfermeiros têm que ter, pelo menos, um dia de folga.

Dado o número N de enfermeiros que se quer assegurar por turno, pretende-se saber qual será o números mínimo de enfermeiros que o hospital deve contratar.

### Exercício 2

Faça a análise do problema acima apresentado. Primeiro, indique claramente as variáveis que vai necessitar e a sua interpretação. Depois, escreva formalmente as restrições que modelam o problema. Por fim, defina o objectivo do problema. 

In [4]:
from ortools.linear_solver import pywraplp

enfermeiro = pywraplp.Solver.CreateSolver('SCIP')
#Salas, Professores,Horas por dia, Dias, Carga Horária máx, mín. aulas por semana
N = 5
D = 7
T = 21 # manhã,tarde,noite num total de 21 ver desenho caderno
E = int(21*N*(1.3))
# turno da manha sera N*1.3
# para contratar será preciso E = 21(turnos na semana) * N*1.3(por excesso, ou seja, no caso dos turnos que tem mais enfermeiros)

#definir as variáveis
# 
x={}
for e in range(E):
    for t in range(T):
        x[e,t]=enfermeiro.BoolVar(f'x[{e}][{t}]')
y={}
for e in range(E):
    for d in range(D):
        y[e,d]= enfermeiro.BoolVar(f'y[{e}][{d}]')

#x[e,d,t] -> determina se um enfermeiro e num certo dia d num turno t está a trabalhar( == 1) ou nao (== 0)
# ou como a stora fez, excluindo os dias visto os turnos serem sempre iguais ao longo dos dias,
# x[e,t] _> dá 1 sse o enfermeiro e se encontra no turno t

1. Todos os turnos têm pelo menos N enfermeiros. 

$ \forall_{t<T} \centerdot \sum_{e<E} x_{e,t} \geq N$

In [5]:
for t in range(T):
    enfermeiro.Add(sum([x[e,t] for e in range (E)]) >= N)

2. Os turnos da manhã, por serem mais movimentados, têm no mínimo, mais 30% de enfermeiros do que é exigido nos restantes turnos.

$ \forall_{t\in{0,3,6,9,12,15,18} } \centerdot \sum_{e<E} x_{e,t} \geq N*1.3 $

In [8]:
listaT = [0,3,6,9,12,15,18]
for t in range (0,len(listaT)):
    enfermeiro.Add(sum([x[e,listaT[t]] for e in range (E)]) >= N*1.3)

3. Um enfermeiro não pode trabalhar dois turnos seguidos

$\forall_{e<E}\forall_{t<T} \ x_{e,t} + x_{e,(t+1)\bmod 21} \le 1$

In [9]:
for e in range(E):
    for t in range(T):
        enfermeiro.Add(x[e,t] + x[e,((t+1)%21)] <= 1)

4. Todos os enfermeiros têm que ter, pelo menos, um dia de folga.

$\forall_{e,E} \cdot \sum_{d<D} y_{e,d} \leq D-1$

In [10]:
for e in range(E):
    enfermeiro.Add(sum([x[e,d] for d in range(D)]) <= D-1)

5. Um enfermeiro só trabalha se tiver um turno num certo dia.

### Exercício 3

Codifique em Python uma função `nurses(N)` que calcula o número mínimo de enfermeiros que um hospital deve contratar.

In [ ]:
# completar